# Unidade Curricular: Processamento de Linguagem Natural


SENAI - Curso Técnico em Inteligência Artificial

Unidade Curricular: Processamento de Linguagem Natural

Data de entrega: 18/10/2021

Docente: Lucas Vilas

Equipe 2 (Olist Reviews Dataset):
*   Iuri
*   Gabriel Trindade
*   Gabriel Trevisan
*   Tácio
*   Rafael


Especificação do projeto :
https://drive.google.com/file/d/1sUAw5pmvGJlMWd5k31tAa14GKLATQIkk/view

Datasets:

https://raw.githubusercontent.com/lucasvbalves/nlp-pt-br-datasets/master/olist-order-reviews-dataset/olist_pt_neg.txt

https://raw.githubusercontent.com/lucasvbalves/nlp-pt-br-datasets/master/olist-order-reviews-dataset/olist_pt_pos.txt

# Importação

Importação das bibliotecas

In [ ]:
!pip install nltk==3.6.2
!pip install spacy==3.1.2
#!pip install stanza==1.2.3

     |████████████████████████████████| 1.5 MB 12.3 MB/s 
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 5.8 MB 11.3 MB/s 
     |████████████████████████████████| 456 kB 71.0 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 627 kB 71.1 MB/s 
     |████████████████████████████████| 10.1 MB 39.3 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing 

In [ ]:
import nltk
import csv
import re
import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
from numpy import arccos, array
from numpy.linalg import norm

import spacy
import spacy.cli
spacy.cli.download("pt_core_news_lg")

# import stanza
# stanza.download('pt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


Importação da dataset utilizando numpy por ser mais compatível.

https://numpy.org/devdocs/reference/generated/numpy.loadtxt.html?highlight=loadtxt

In [ ]:
filename = 'https://raw.githubusercontent.com/lucasvbalves/nlp-pt-br-datasets/master/olist-order-reviews-dataset/olist_pt_neg.txt'
datan = np.loadtxt(filename, delimiter='\n', dtype=str)
filename = 'https://raw.githubusercontent.com/lucasvbalves/nlp-pt-br-datasets/master/olist-order-reviews-dataset/olist_pt_pos.txt'
datap = np.loadtxt(filename, delimiter='\n', dtype=str)

# Visualização do dataset/texto

In [ ]:
data = np.loadtxt(filename, delimiter='\n', dtype=str, encoding='bytes')
data

array(['A única dica para a loja é que otimizem o envio, comprei 3 frascos e cada um veio em uma caixa. o tamanho de uma caixa cabia os 3 frascos de vidro. o produto é ótimo.',
       'São inúmeras as vezes a qual comprei e continuo a comprar com o stark recomendo sempre com preços excelentes entrega dentro do prazo e entregue em perfeito estado',
       'Produto entregue sem defeito e dentro do prazo, mas o produto não é bom; a base é de plástico e muito ruim, as argolas são ruins também.',
       'Preço muito convidativo, entrega no prazo,produto bem embalado, mas deveria haver mais cuidado na embalagem, produto veio solto na caixa, poderia vir com protetores para evitar danos, que não houveram',
       'Eu comprei uma bicicleta GTS M1 só que veio outra bicicleta fora que o canote não entra no quadro o quadro veio riscado e totalmente diferente do produto que adquiri eu só vou recomendar a loja',
       'eu recebi produto, mais a cor veio errada, eu pedi preto preto, e veio vermelho,

In [ ]:
datap_df = pd.DataFrame(datap,columns=['Texto'])
datan_df = pd.DataFrame(datan,columns=['Texto'])

In [ ]:
datap_df

,Texto
0,A única dica para a loja é que otimizem o envi...
1,São inúmeras as vezes a qual comprei e continu...
2,Produto entregue sem defeito e dentro do prazo...
3,"Preço muito convidativo, entrega no prazo,prod..."
4,Eu comprei uma bicicleta GTS M1 só que veio ou...
...,...
995,"Produto de material de boa qualidade, porém as..."
996,A princípio recebi um e fiquei chateada pois o...
997,eu gostei muito de comprar pelo sait é confial...
998,Porém achei o frete caro e veio o sabor errado...


In [ ]:
danp_df = datap_df.append(datan_df).dropna()
danp_df = danp_df.sample(frac=1,random_state=1).reset_index(drop=True)
danp_df = danp_df.astype({ 'Texto' : 'string'})

In [ ]:
danp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Texto   2000 non-null   string
dtypes: string(1)
memory usage: 15.8 KB


In [ ]:
danp_df

,Texto
0,"Entrega e qualidade excelente, porém quanto ao..."
1,Olha estou muito decepcionado pq sempre compre...
2,A entrega até o presente não ocorreu e já abri...
3,"Ainda não recebi meu produto, espero uma respo..."
4,Pedi duas cubas so recebi uma estou aguardando...
...,...
1995,Desta vez meu produto não foi entregue correta...
1996,A capinha veio completamente danificada e rasu...
1997,comprei um conjunto de skate infantil masculin...
1998,"Receber um produto de excelente qualidade, com..."


In [ ]:
clone = danp_df['Texto'].is_unique
clones = danp_df['Texto'].duplicated().any()
print(clone,"e",clones)

True e False


# Definindo algumas strings

In [ ]:
reviews = danp_df[danp_df['Texto'].str.contains('produto')]
review1 = reviews.iloc[0, 0]
review2 = reviews.iloc[9, 0]

In [ ]:
reviews

,Texto
1,Olha estou muito decepcionado pq sempre compre...
3,"Ainda não recebi meu produto, espero uma respo..."
6,"Produto de ótima qualidade, entrega super rápi..."
7,O que seria mais fácil seria fazer uma nota só...
9,Obrigada pela rapidez! Fiquei muito satisfeita...
...,...
1994,"Adoro comprar aqui. Tenho o que necessito, em ..."
1995,Desta vez meu produto não foi entregue correta...
1996,A capinha veio completamente danificada e rasu...
1998,"Receber um produto de excelente qualidade, com..."


In [ ]:
review = danp_df.iloc[1026, 0]

In [ ]:
print(review)

Comprei com um pouco de receio de ficar muito escuro, pois meu cabelo esta castanho escuro e tem uns tons meio marrom. Porém deu uma leva mudada na cor mais n ficou "pretão" deu mto brilho e amei 😍


# Tokenize

In [ ]:
tokens = word_tokenize(review, language='portuguese')
tokens_numbers = Counter(tokens)

In [ ]:
tokens_numbers

Counter({"''": 1,
         ',': 1,
         '.': 1,
         'Comprei': 1,
         'Porém': 1,
         '``': 1,
         'amei': 1,
         'brilho': 1,
         'cabelo': 1,
         'castanho': 1,
         'com': 1,
         'cor': 1,
         'de': 2,
         'deu': 2,
         'e': 2,
         'escuro': 2,
         'esta': 1,
         'ficar': 1,
         'ficou': 1,
         'leva': 1,
         'mais': 1,
         'marrom': 1,
         'meio': 1,
         'meu': 1,
         'mto': 1,
         'mudada': 1,
         'muito': 1,
         'n': 1,
         'na': 1,
         'pois': 1,
         'pouco': 1,
         'pretão': 1,
         'receio': 1,
         'tem': 1,
         'tons': 1,
         'um': 1,
         'uma': 1,
         'uns': 1,
         '😍': 1})

# Tokenize do dataset

In [ ]:
danp = danp_df.iloc[0:2000, 0]
danp = danp.tolist()

In [ ]:
danp = re.sub(r'[^\w\s]', '', (str(danp)))
danp = danp.lower()
print(danp)

entrega e qualidade excelente porém quanto ao anúncio para mim foi surpresa receber uma coqueteleira pois esperava dois potes de creatina como o este me levou a crer olha estou muito decepcionado pq sempre comprei pelo site e sempre veio os produtos certinhos agora foi uma sacanagem que fizeram comigo anunciaram a foto de um produto é mandaram outro a entrega até o presente não ocorreu e já abri chamado no reclame aqui e ninguém me retorna ao meu ver uma grande falta de respeito com o cliente  aguardo retorno urgente ainda não recebi meu produto espero uma resposta de vcs ou a devolução de um vale para fazer outra compra já era pra ter sido entregue ontem dia 110418 é nada ainda fico aguardando uma respost pedi duas cubas so recebi uma estou aguardando resposta de como posso resolver isso no mais a entrega foi no prazo e a que recebi e de boa qualidade olá meu pedido não foi entregue quero saber o que aconteceu se não for resolvido eu vou querer o meu dinheiro de volta agradeço desde j

In [ ]:
tokens = word_tokenize(danp, language='portuguese')
tokens = [word for word in tokens if not word in stopwords.words('portuguese')]
# stopword_list = ['vc','n','m','pq','s','qdo','hj','p','nd','d']
# tokens = [word for word in tokens if not word in stopword_list]
tokens_numbers = Counter(tokens)
tokens_numbers

Counter({'entrega': 485,
         'qualidade': 162,
         'excelente': 56,
         'porém': 167,
         'quanto': 30,
         'anúncio': 27,
         'mim': 33,
         'surpresa': 13,
         'receber': 72,
         'coqueteleira': 1,
         'pois': 228,
         'esperava': 20,
         'dois': 109,
         'potes': 3,
         'creatina': 2,
         'levou': 5,
         'crer': 2,
         'olha': 9,
         'decepcionado': 15,
         'pq': 47,
         'sempre': 134,
         'comprei': 429,
         'site': 199,
         'veio': 369,
         'produtos': 170,
         'certinhos': 1,
         'agora': 143,
         'sacanagem': 3,
         'fizeram': 9,
         'comigo': 15,
         'anunciaram': 1,
         'foto': 71,
         'produto': 1356,
         'é': 741,
         'mandaram': 31,
         'outro': 100,
         'presente': 48,
         'ocorreu': 24,
         'abri': 13,
         'chamado': 1,
         'reclame': 4,
         'aqui': 27,
         'ninguém

In [ ]:
stopwords.words('portuguese')

# Lematização

In [ ]:
rslp_stemmer = nltk.stem.RSLPStemmer()
snowball_stemmer = nltk.stem.snowball.SnowballStemmer(language='portuguese')

In [ ]:
danp_words = []
rslp_stems = []
snowball_stems = []

print('{:20}{:20}{:20}'.format('PALAVRA', 'RSLP Stemmer', 'Snowball Stemmer'))
print('----------------------------------------------------------------------')

for i, token in enumerate(tokens):

    rslp_stem = rslp_stemmer.stem(token)
    snowball_stem = snowball_stemmer.stem(token)

    print('{:20}{:20}{:20}'.format(token, rslp_stem, snowball_stem))

    danp_words.append(token)
    rslp_stems.append(rslp_stem)
    snowball_stems.append(snowball_stem)

Streaming output truncated to the last 5000 lines.
infelizmente        infeliz             infeliz             
sei                 sei                 sei                 
problema            problem             problem             
correios            correi              correi              
loja                loj                 loj                 
fez                 fez                 fez                 
algo                alg                 algo                
errado              err                 errad               
remessa             remess              remess              
primeira            prim                primeir             
entrega             entreg              entreg              
película            películ             películ             
vidro               vidr                vidr                
veio                vei                 vei                 
trincada            trinc               trinc               
após                após          

In [ ]:
print('Tokens: ',len(rslp_stems))
print('Vocabulário: ',len(set(rslp_stems)))
print('Tokens: ',len(snowball_stems))
print('Vocabulário: ',len(set(snowball_stems)))

Tokens:  36759
Vocabulário:  4040
Tokens:  36759
Vocabulário:  4309


In [ ]:
nlp = spacy.load('pt_core_news_lg')

In [ ]:
spacy_words = []
spacy_lemmas = []

print('{:20}{:20}'.format('PALAVRA', 'Spacy Lemmatizer'))
print('------------------------------------')

for token in nlp(danp):

    word = token.text
    word_lemma = token.lemma_

    print('{:20}{:20}'.format(
        word, word_lemma)
    )

    spacy_words.append(word)
    spacy_lemmas.append(word_lemma)

Streaming output truncated to the last 5000 lines.
site                site                
foram               ser                 
entregues           entregue            
em                  em                  
uma                 umar                
semana              semana              
apesar              apesar              
da                  da                  
demora              demorar             
nessa               nessa               
entrega             entregar            
recomendo           recomendar          
o                   o                   
site                site                
produto             produto             
ja                  ja                  
foi                 ser                 
devolvido           devolver            
1                   1                   
vez                 vez                 
por                 por                 
entrega             entregar            
com                 com                 
peças 

In [ ]:
print('Tokens: ',len(spacy_lemmas))
print('Vocabulário: ',len(set(spacy_lemmas)))

Tokens:  63393
Vocabulário:  4675


In [ ]:
letokens_numbers = Counter(spacy_lemmas)

In [ ]:
letokens_numbers

Counter({'entregar': 921,
         'e': 2101,
         'qualidade': 162,
         'excelente': 61,
         'porém': 167,
         'quantum': 30,
         'ao': 141,
         'anúncio': 27,
         'parir': 584,
         'mim': 33,
         'ser': 2036,
         'surpreso': 13,
         'receber': 621,
         'umar': 508,
         'coqueteleira': 1,
         'pois': 228,
         'esperar': 136,
         'dois': 189,
         'pote': 4,
         'de': 1728,
         'creatina': 2,
         'comer': 231,
         'o': 6124,
         'este': 109,
         'me': 281,
         'levar': 29,
         'crer': 10,
         'olhar': 16,
         'estar': 629,
         'muito': 509,
         'decepcionar': 46,
         'pq': 47,
         'sempre': 134,
         'comprar': 1025,
         'pelar': 258,
         'site': 201,
         'vir': 486,
         'produto': 1526,
         'certo': 32,
         'agora': 143,
         'sacanagem': 3,
         'que': 1574,
         'fazer': 304,
         'c

# Martriz da análise

In [ ]:
danptk = re.sub(r'[^\w\s]', '', (str(tokens)))
danptk = danptk.lower()
danpsl = re.sub(r'[^\w\s]', '', (str(spacy_lemmas)))
danpsl = danpsl.lower()

In [ ]:
corpus = [danptk,danpsl]

In [ ]:
covecto = CountVectorizer(lowercase=True)
covecto.fit(corpus)
wordsv1 = covecto.vocabulary_
vector1 = covecto.transform(corpus)

In [ ]:
print(vector1.shape)
type(vector1)
vector1.toarray()

(2, 7529)


array([[ 1,  1,  1, ..., 12,  1,  4],
       [ 1,  1,  1, ...,  0,  1, 16]])

In [ ]:
terms1 = [vocab_tuple[0] for vocab_tuple in sorted(wordsv1.items(), key=lambda item: item[1])]
bag1_df = pd.DataFrame(vector1.toarray(), columns=terms1)

In [ ]:
tfvecto = TfidfVectorizer()
tfvecto.fit(corpus)
wordsv2 = tfvecto.vocabulary_
vector2 = tfvecto.transform(corpus)

In [ ]:
vector2.shape
type(vector2)
vector2.toarray()

array([[0.00040513, 0.00040513, 0.00040513, ..., 0.00683274, 0.00040513,
        0.00162052],
       [0.00016174, 0.00016174, 0.00016174, ..., 0.        , 0.00016174,
        0.00258788]])

In [ ]:
terms2 = [vocab_tuple[0] for vocab_tuple in sorted(wordsv2.items(), key=lambda item: item[1])]
bag2_df = pd.DataFrame(vector2.toarray(), columns=terms2)

# CountVectorizer

In [ ]:
covecto = CountVectorizer(lowercase=True)
corpus = [review1, review2]
covecto.fit(corpus)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
wordsv1 = covecto.vocabulary_

In [ ]:
wordsv1

{'2007': 0,
 'agora': 1,
 'amigos': 2,
 'anunciaram': 3,
 'aos': 4,
 'baratheon': 5,
 'certinhos': 6,
 'cliente': 7,
 'comigo': 8,
 'comprei': 9,
 'de': 10,
 'decepcionado': 11,
 'desde': 12,
 'do': 13,
 'em': 14,
 'estou': 15,
 'familiares': 16,
 'faço': 17,
 'feliz': 18,
 'fico': 19,
 'fizeram': 20,
 'foi': 21,
 'foto': 22,
 'mandaram': 23,
 'muito': 24,
 'olha': 25,
 'os': 26,
 'outro': 27,
 'pelo': 28,
 'possível': 29,
 'pq': 30,
 'produto': 31,
 'produtos': 32,
 'propaganda': 33,
 'que': 34,
 'receber': 35,
 'sacanagem': 36,
 'sempre': 37,
 'site': 38,
 'sou': 39,
 'um': 40,
 'uma': 41,
 'veio': 42,
 'vocês': 43}

In [ ]:
vector1 = covecto.transform(corpus)
print(vector1.shape)
type(vector1)
vector1.toarray()

(2, 44)


array([[0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 2, 1, 0, 1, 1, 1, 0],
       [1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 2, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 2]])

In [ ]:
terms1 = [vocab_tuple[0] for vocab_tuple in sorted(wordsv1.items(), key=lambda item: item[1])]
bag1 = pd.DataFrame(vector1.toarray(), columns=terms1)

# TfidfVectorizer

In [ ]:
tfvecto = TfidfVectorizer()
tfvecto.fit(corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
wordsv2 = tfvecto.vocabulary_

In [ ]:
wordsv2

{'2007': 0,
 'agora': 1,
 'amigos': 2,
 'anunciaram': 3,
 'aos': 4,
 'baratheon': 5,
 'certinhos': 6,
 'cliente': 7,
 'comigo': 8,
 'comprei': 9,
 'de': 10,
 'decepcionado': 11,
 'desde': 12,
 'do': 13,
 'em': 14,
 'estou': 15,
 'familiares': 16,
 'faço': 17,
 'feliz': 18,
 'fico': 19,
 'fizeram': 20,
 'foi': 21,
 'foto': 22,
 'mandaram': 23,
 'muito': 24,
 'olha': 25,
 'os': 26,
 'outro': 27,
 'pelo': 28,
 'possível': 29,
 'pq': 30,
 'produto': 31,
 'produtos': 32,
 'propaganda': 33,
 'que': 34,
 'receber': 35,
 'sacanagem': 36,
 'sempre': 37,
 'site': 38,
 'sou': 39,
 'um': 40,
 'uma': 41,
 'veio': 42,
 'vocês': 43}

In [ ]:
vector2 = tfvecto.transform(corpus)
vector2.shape
type(vector2)
vector2.toarray()

array([[0.        , 0.19781166, 0.        , 0.19781166, 0.        ,
        0.        , 0.19781166, 0.        , 0.19781166, 0.19781166,
        0.14074463, 0.19781166, 0.        , 0.        , 0.        ,
        0.19781166, 0.        , 0.        , 0.        , 0.        ,
        0.19781166, 0.19781166, 0.19781166, 0.19781166, 0.14074463,
        0.19781166, 0.14074463, 0.19781166, 0.19781166, 0.        ,
        0.19781166, 0.19781166, 0.14074463, 0.        , 0.14074463,
        0.        , 0.19781166, 0.28148926, 0.19781166, 0.        ,
        0.19781166, 0.19781166, 0.19781166, 0.        ],
       [0.20179921, 0.        , 0.20179921, 0.        , 0.20179921,
        0.20179921, 0.        , 0.20179921, 0.        , 0.        ,
        0.2871636 , 0.        , 0.20179921, 0.20179921, 0.20179921,
        0.        , 0.20179921, 0.20179921, 0.20179921, 0.20179921,
        0.        , 0.        , 0.        , 0.        , 0.1435818 ,
        0.        , 0.1435818 , 0.        , 0.        , 0.2

In [ ]:
terms2 = [vocab_tuple[0] for vocab_tuple in sorted(wordsv2.items(), key=lambda item: item[1])]
bag2 = pd.DataFrame(vector2.toarray(), columns=terms2)

# Os dois vetores

In [ ]:
bag1

,2007,agora,amigos,anunciaram,aos,baratheon,certinhos,cliente,comigo,comprei,de,decepcionado,desde,do,em,estou,familiares,faço,feliz,fico,fizeram,foi,foto,mandaram,muito,olha,os,outro,pelo,possível,pq,produto,produtos,propaganda,que,receber,sacanagem,sempre,site,sou,um,uma,veio,vocês
0,0,1,0,1,0,0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,2,1,0,1,1,1,0
1,1,0,1,0,1,1,0,1,0,0,2,0,1,1,1,0,1,1,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,1,1,0,1,0,1,0,0,0,2


In [ ]:
bag2

,2007,agora,amigos,anunciaram,aos,baratheon,certinhos,cliente,comigo,comprei,de,decepcionado,desde,do,em,estou,familiares,faço,feliz,fico,fizeram,foi,foto,mandaram,muito,olha,os,outro,pelo,possível,pq,produto,produtos,propaganda,que,receber,sacanagem,sempre,site,sou,um,uma,veio,vocês
0,0.000000,0.197812,0.000000,0.197812,0.000000,0.000000,0.197812,0.000000,0.197812,0.197812,0.140745,0.197812,0.000000,0.000000,0.000000,0.197812,0.000000,0.000000,0.000000,0.000000,0.197812,0.197812,0.197812,0.197812,0.140745,0.197812,0.140745,0.197812,0.197812,0.000000,0.197812,0.197812,0.140745,0.000000,0.140745,0.000000,0.197812,0.281489,0.197812,0.000000,0.197812,0.197812,0.197812,0.000000
1,0.201799,0.000000,0.201799,0.000000,0.201799,0.201799,0.000000,0.201799,0.000000,0.000000,0.287164,0.000000,0.201799,0.201799,0.201799,0.000000,0.201799,0.201799,0.201799,0.201799,0.000000,0.000000,0.000000,0.000000,0.143582,0.000000,0.143582,0.000000,0.000000,0.201799,0.000000,0.000000,0.143582,0.201799,0.143582,0.201799,0.000000,0.143582,0.000000,0.201799,0.000000,0.000000,0.000000,0.403598


In [ ]:
bag1_df

,000000000000000000000000000,000024453,003570,01,01092017,012495,0168695364,01um,02,0201,02082018,0213,02658094288,02658697704,02660437435,02666377620,03,0303,03032018,0304,03042018,0305,0307,0311,03500100047,03500887458,04,04072017,04072018,041016,05,05012018,05092017,05cm,06032018,0606,06062017,06062018,060818,0612,...,zenfone,zero,zíper,às,ágil,água,área,ártico,árvore,ânimo,ão,ée,época,étic,êxito,íntegras,íntegro,ítem,óculo,óculos,óleo,órgão,ótima,ótimas,ótimo,ótimoa,ótimos,ônibusrua,última,últimas,último,últimos,úmida,única,únicar,único,únicos,úteis,úteisesperei,útil
0,1,1,1,18,1,1,1,2,13,1,1,1,1,1,1,1,7,1,1,1,1,1,1,1,1,1,2,1,1,1,4,1,1,1,1,2,1,1,1,1,...,2,1,3,0,2,13,10,1,1,1,1,1,3,1,3,1,0,1,0,5,2,1,32,2,54,1,7,1,7,2,5,1,1,23,1,36,1,12,1,4
1,1,1,1,18,1,1,1,2,13,1,1,1,1,1,1,1,7,1,1,1,1,1,1,1,1,1,2,1,1,1,5,1,1,0,1,2,1,1,1,1,...,2,1,3,10,2,13,10,1,1,1,1,1,3,1,3,0,1,1,5,0,2,1,32,2,54,1,7,1,0,0,15,0,1,0,1,60,0,0,1,16


In [ ]:
bag2_df

,000000000000000000000000000,000024453,003570,01,01092017,012495,0168695364,01um,02,0201,02082018,0213,02658094288,02658697704,02660437435,02666377620,03,0303,03032018,0304,03042018,0305,0307,0311,03500100047,03500887458,04,04072017,04072018,041016,05,05012018,05092017,05cm,06032018,0606,06062017,06062018,060818,0612,...,zenfone,zero,zíper,às,ágil,água,área,ártico,árvore,ânimo,ão,ée,época,étic,êxito,íntegras,íntegro,ítem,óculo,óculos,óleo,órgão,ótima,ótimas,ótimo,ótimoa,ótimos,ônibusrua,última,últimas,último,últimos,úmida,única,únicar,único,únicos,úteis,úteisesperei,útil
0,0.000405,0.000405,0.000405,0.007292,0.000405,0.000405,0.000405,0.000810,0.005267,0.000405,0.000405,0.000405,0.000405,0.000405,0.000405,0.000405,0.002836,0.000405,0.000405,0.000405,0.000405,0.000405,0.000405,0.000405,0.000405,0.000405,0.000810,0.000405,0.000405,0.000405,0.001621,0.000405,0.000405,0.000569,0.000405,0.000810,0.000405,0.000405,0.000405,0.000405,...,0.000810,0.000405,0.001215,0.000000,0.000810,0.005267,0.004051,0.000405,0.000405,0.000405,0.000405,0.000405,0.001215,0.000405,0.001215,0.000569,0.000000,0.000405,0.000000,0.002847,0.000810,0.000405,0.012964,0.000810,0.021877,0.000405,0.002836,0.000405,0.003986,0.001139,0.002026,0.000569,0.000405,0.013096,0.000405,0.014585,0.000569,0.006833,0.000405,0.001621
1,0.000162,0.000162,0.000162,0.002911,0.000162,0.000162,0.000162,0.000323,0.002103,0.000162,0.000162,0.000162,0.000162,0.000162,0.000162,0.000162,0.001132,0.000162,0.000162,0.000162,0.000162,0.000162,0.000162,0.000162,0.000162,0.000162,0.000323,0.000162,0.000162,0.000162,0.000809,0.000162,0.000162,0.000000,0.000162,0.000323,0.000162,0.000162,0.000162,0.000162,...,0.000323,0.000162,0.000485,0.002273,0.000323,0.002103,0.001617,0.000162,0.000162,0.000162,0.000162,0.000162,0.000485,0.000162,0.000485,0.000000,0.000227,0.000162,0.001137,0.000000,0.000323,0.000162,0.005176,0.000323,0.008734,0.000162,0.001132,0.000162,0.000000,0.000000,0.002426,0.000000,0.000162,0.000000,0.000162,0.009705,0.000000,0.000000,0.000162,0.002588


# Cosseno do Count

In [ ]:
count_vect = CountVectorizer()
sparse_doc_term_matrix = count_vect.fit_transform(corpus)

In [ ]:
doc_term_matrix = sparse_doc_term_matrix.todense()
doc1_df = pd.DataFrame(doc_term_matrix,
    columns=count_vect.get_feature_names(),
    index=['review1', 'review2'])

In [ ]:
similarity_array = cosine_similarity(doc1_df)
similarity1_df = pd.DataFrame(similarity_array,
    columns=['review1', 'review2'],
    index=['review1', 'review2'])

In [ ]:
cosine_dist_array = cosine_distances(doc1_df)
cosine1_df = pd.DataFrame(cosine_dist_array,
    columns=['review1', 'review2'],
    index=['review1', 'review2'])

# Cosseno do Tfidf

In [ ]:
tfidf_vect = TfidfVectorizer()
sparse_doc_term_matrix = tfidf_vect.fit_transform(corpus)

In [ ]:
doc_term_matrix = sparse_doc_term_matrix.todense()
doc2_df = pd.DataFrame(doc_term_matrix,
    columns=tfidf_vect.get_feature_names(),
    index=['review1', 'review2'])

In [ ]:
similarity_array = cosine_similarity(doc2_df)
similarity2_df = pd.DataFrame(similarity_array,
    columns=['review1', 'review2'],
    index=['review1', 'review2'])

In [ ]:
cosine_dist_array = cosine_distances(doc2_df)
cosine2_df = pd.DataFrame(cosine_dist_array,
    columns=['review1', 'review2'],
    index=['review1', 'review2'])

# Diferenças entre o cosseno

In [ ]:
doc1_df

,2007,agora,amigos,anunciaram,aos,baratheon,certinhos,cliente,comigo,comprei,de,decepcionado,desde,do,em,estou,familiares,faço,feliz,fico,fizeram,foi,foto,mandaram,muito,olha,os,outro,pelo,possível,pq,produto,produtos,propaganda,que,receber,sacanagem,sempre,site,sou,um,uma,veio,vocês
review1,0,1,0,1,0,0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,2,1,0,1,1,1,0
review2,1,0,1,0,1,1,0,1,0,0,2,0,1,1,1,0,1,1,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,1,1,0,1,0,1,0,0,0,2


In [ ]:
similarity1_df

,review1,review2
review1,1.000000,0.271225
review2,0.271225,1.000000


In [ ]:
cosine1_df

,review1,review2
review1,0.000000,0.728775
review2,0.728775,0.000000


In [ ]:
doc2_df

,2007,agora,amigos,anunciaram,aos,baratheon,certinhos,cliente,comigo,comprei,de,decepcionado,desde,do,em,estou,familiares,faço,feliz,fico,fizeram,foi,foto,mandaram,muito,olha,os,outro,pelo,possível,pq,produto,produtos,propaganda,que,receber,sacanagem,sempre,site,sou,um,uma,veio,vocês
review1,0.000000,0.197812,0.000000,0.197812,0.000000,0.000000,0.197812,0.000000,0.197812,0.197812,0.140745,0.197812,0.000000,0.000000,0.000000,0.197812,0.000000,0.000000,0.000000,0.000000,0.197812,0.197812,0.197812,0.197812,0.140745,0.197812,0.140745,0.197812,0.197812,0.000000,0.197812,0.197812,0.140745,0.000000,0.140745,0.000000,0.197812,0.281489,0.197812,0.000000,0.197812,0.197812,0.197812,0.000000
review2,0.201799,0.000000,0.201799,0.000000,0.201799,0.201799,0.000000,0.201799,0.000000,0.000000,0.287164,0.000000,0.201799,0.201799,0.201799,0.000000,0.201799,0.201799,0.201799,0.201799,0.000000,0.000000,0.000000,0.000000,0.143582,0.000000,0.143582,0.000000,0.000000,0.201799,0.000000,0.000000,0.143582,0.201799,0.143582,0.201799,0.000000,0.143582,0.000000,0.201799,0.000000,0.000000,0.000000,0.403598


In [ ]:
similarity2_df

,review1,review2
review1,1.000000,0.161667
review2,0.161667,1.000000


In [ ]:
cosine2_df

,review1,review2
review1,0.000000,0.838333
review2,0.838333,0.000000
